In [1]:
import numpy as np
import cv2
from numpy.core.fromnumeric import trace
import mediapipe_utils as mpu
from pathlib import Path
from FPS import FPS, now
import depthai as dai
import marshal
import sys
from string import Template
from math import sin, cos
from BlazeposeDepthaiEdge import BlazeposeDepthai
from BlazeposeRenderer import BlazeposeRenderer


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Importan Landmarks
- 11 : Left Shoulder


In [2]:
important_landmarks = ['left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index']
important_landmarks = [mpu.KEYPOINT_DICT[l] for l in important_landmarks]

In [3]:
tracker = BlazeposeDepthai(
                input_src="rgb",
                pd_model=None, 
                pd_score_thresh=0.5,
                pp_model=None,
                lm_model=None,
                lm_score_thresh=0.7,
                xyz = True,
                crop=False,
                smoothing= True,
                filter_window_size=5,
                filter_velocity_scale=10,
                stats=False,               
                internal_fps=None,
                internal_frame_height=1080,
                trace=False,
                force_detection=False)   

renderer = BlazeposeRenderer(
                tracker)
#Reset coordernates
LEFT_SHOULDER = None
RIGHT_SHOULDER = None
LEFT_ELBOW = None
RIGHT_ELBOW = None
LEFT_WRIST = None
RIGHT_WRIST = None
LEFT_HIP = None
RIGHT_HIP = None
LEFT_KNEE = None
RIGHT_KNEE = None
LEFT_ANKLE = None
RIGHT_ANKLE = None

while True:
    try:
        # Run blazepose on next frame
        frame, body = tracker.next_frame()
        if frame is None: break
        # Draw 2d skeleton
        frame = renderer.draw(frame, body, my_keypoints=important_landmarks)
        key = renderer.waitKey(delay=1)
        
        # # Get 3d coordinates
        # if body:
        #     LEFT_SHOULDER = body.landmarks[mpu.KEYPOINT_DICT['left_shoulder']] if renderer.is_present(body, mpu.KEYPOINT_DICT['left_shoulder']) else None
        #     RIGHT_SHOULDER = body.landmarks[mpu.KEYPOINT_DICT['right_shoulder']] if renderer.is_present(body, mpu.KEYPOINT_DICT['right_shoulder']) else None
        #     LEFT_ELBOW = body.landmarks[mpu.KEYPOINT_DICT['left_elbow']] if renderer.is_present(body, mpu.KEYPOINT_DICT['left_elbow']) else None
        #     RIGHT_ELBOW = body.landmarks[mpu.KEYPOINT_DICT['right_elbow']] if renderer.is_present(body, mpu.KEYPOINT_DICT['right_elbow']) else None
        #     LEFT_WRIST = body.landmarks[mpu.KEYPOINT_DICT['left_wrist']] if renderer.is_present(body, mpu.KEYPOINT_DICT['left_wrist']) else None
        #     RIGHT_WRIST = body.landmarks[mpu.KEYPOINT_DICT['right_wrist']] if renderer.is_present(body, mpu.KEYPOINT_DICT['right_wrist']) else None
        #     LEFT_HIP = body.landmarks[mpu.KEYPOINT_DICT['left_hip']] if renderer.is_present(body, mpu.KEYPOINT_DICT['left_hip']) else None
        #     RIGHT_HIP = body.landmarks[mpu.KEYPOINT_DICT['right_hip']] if renderer.is_present(body, mpu.KEYPOINT_DICT['right_hip']) else None
        #     LEFT_KNEE = body.landmarks[mpu.KEYPOINT_DICT['left_knee']] if renderer.is_present(body, mpu.KEYPOINT_DICT['left_knee']) else None
        #     RIGHT_KNEE = body.landmarks[mpu.KEYPOINT_DICT['right_knee']] if renderer.is_present(body, mpu.KEYPOINT_DICT['right_knee']) else None
        #     LEFT_ANKLE = body.landmarks[mpu.KEYPOINT_DICT['left_ankle']] if renderer.is_present(body, mpu.KEYPOINT_DICT['left_ankle']) else None
        #     RIGHT_ANKLE = body.landmarks[mpu.KEYPOINT_DICT['right_ankle']] if renderer.is_present(body, mpu.KEYPOINT_DICT['right_ankle']) else None
        #     LEFT_HEEL = body.landmarks[mpu.KEYPOINT_DICT['left_heel']] if renderer.is_present(body, mpu.KEYPOINT_DICT['left_heel']) else None
        #     RIGHT_HEEL = body.landmarks[mpu.KEYPOINT_DICT['right_heel']] if renderer.is_present(body, mpu.KEYPOINT_DICT['right_heel']) else None
        #     LEFT_FOOT_INDEX = body.landmarks[mpu.KEYPOINT_DICT['left_foot_index']] if renderer.is_present(body, mpu.KEYPOINT_DICT['left_foot_index']) else None
        #     RIGHT_FOOT_INDEX = body.landmarks[mpu.KEYPOINT_DICT['right_foot_index']] if renderer.is_present(body, mpu.KEYPOINT_DICT['right_foot_index']) else None
        
        
            
        if key == 27 or key == ord('q'):
            break
    except Exception as e:
        print(e)
        break
renderer.exit()
tracker.exit()
cv2.destroyAllWindows()

Pose detection blob file : D:\PythonComputing\StrongAI\models\pose_detection_sh4.blob
Landmarks using blob file : D:\PythonComputing\StrongAI\models\pose_landmark_full_sh4.blob
Internal camera FPS set to: 18
Internal camera image size: 1792 x 1008 - pad_h: 392
Creating pipeline...
Creating Color Camera...
Creating MonoCameras, Stereo and SpatialLocationCalculator nodes...
RGB calibration lens position: 0
Creating Pose Detection pre processing image manip...
Creating Pose Detection Neural Network...
Creating Pose Detection post processing Neural Network...
Creating Landmark pre processing image manip...
Creating DiveideBy255 Neural Network...
Creating Landmark Neural Network...
Pipeline created.
Pipeline started - USB speed: HIGH
